In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('white')  # plot formatting

mushroom dataset, classify as poisonous or edible :D

In [2]:
columns=["edible?", "cap-shape", "cap-surface", "cap-color", "bruises?", "odor", "gill-attachment",
        "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring",
        "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", 
        "veil-color", "ring-number", "ring-type", "spore-print-color", "population", "habitat"]
df = pd.read_csv("datasets/mushroom.data", header=0, names=columns, index_col=False)
df.head()

,edible?,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
1,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
2,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
3,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
4,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [3]:
df.shape

(8123, 23)

In [4]:
df.isnull().sum()

edible?                     0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises?                    0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [5]:
df['edible?'].value_counts() #pretty even dataset

e    4208
p    3915
Name: edible?, dtype: int64

In [6]:
def standardizeEdibility(letter):
    """
    1: means its edible :D
    0: DONT EAT IT!
    """
    if letter == 'e':
        return 1
    else:
        return 0
df['edible?'] = df['edible?'].apply(standardizeEdibility)

In [7]:
df['edible?'].value_counts()

1    4208
0    3915
Name: edible?, dtype: int64

In [8]:
X = df.drop(['edible?'], axis=1)
y = df['edible?']
X.head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
1,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
2,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
3,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g
4,x,y,y,t,a,f,c,b,n,e,...,s,w,w,p,w,o,p,k,n,g


In [9]:
X_ohe = pd.get_dummies(X)
X_ohe.head()#one hot encoding it

,cap-shape_b,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_f,cap-surface_g,cap-surface_s,cap-surface_y,...,population_s,population_v,population_y,habitat_d,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0


In [10]:
X_le = X.apply(LabelEncoder().fit_transform)#label encode for random forest
X_le.head()

,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,5,2,9,1,0,1,0,0,4,0,...,2,7,7,0,2,1,4,3,2,1
1,0,2,8,1,3,1,0,0,5,0,...,2,7,7,0,2,1,4,3,2,3
2,5,3,8,1,6,1,0,1,5,0,...,2,7,7,0,2,1,4,2,3,5
3,5,2,3,0,5,1,1,0,4,1,...,2,7,7,0,2,1,0,3,0,1
4,5,3,9,1,0,1,0,0,5,0,...,2,7,7,0,2,1,4,2,2,1


# Algorithm 1: Logistic Regression

In [11]:
def LG_model(X, y):
    '''returns the scores of each trial'''
    
    pipe = Pipeline(steps=[('classifier', LogisticRegression())])

    search_space = [{'classifier': [LogisticRegression(max_iter=5000)],
                     'classifier__solver': ['saga'],
                     'classifier__penalty': ['l1', 'l2'],
                     'classifier__C': np.logspace(-8, 4, 13)},
                    {'classifier': [LogisticRegression(max_iter=5000)],
                     'classifier__solver': ['lbfgs'],
                     'classifier__penalty': ['l2'],
                     'classifier__C': np.logspace(-8, 4, 13)},
                    {'classifier': [LogisticRegression(max_iter=5000)],
                     'classifier__solver': ['lbfgs','saga'],
                     'classifier__penalty': ['none']}
                    ]
    
    TRIALS = 5#DOING 5 TRIALS
    log_test_acc = []#saving the accuracy for each logistic regression trial on test set
    log_test_auc = []#saving the roc auc for each logistic regression trial on test set
    log_test_f1 = []#saving the f1 score for each logistic regression trial on test set
    train_acc = []
    train_auc = []
    train_f1 = []

    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        
        # Create grid search |
        clf = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                       verbose=1, n_jobs=-1)
        best_model = clf.fit(X_train, y_train)

        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1'])]

        model_params = [model_acc, model_auc, model_f1]

        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            C = 0
            try:
                C = params['classifier__C']
            except KeyError:
               pass

            solver = params['classifier__solver']
            penalty = params['classifier__penalty']

            #training best models on validation set
            logit_model = LogisticRegression(max_iter=5000)
            logit_model.set_params(C=C, solver=solver, penalty=penalty)
            logit_model.fit(X_train, y_train) 

            #get roc_auc, acc and f1 scores on test set
            if index == 0:
                score = logit_model.score(X_test, y_test)
                log_test_acc.append(score)
                train_score = logit_model.score(X_train, y_train)
                train_acc.append(train_score)

            elif index == 1:
                roc_score = roc_auc_score(y_test, logit_model.predict_proba(X_test)[:, 1])
                log_test_auc.append(roc_score)
                train_roc = roc_auc_score(y_train, logit_model.predict_proba(X_train)[:, 1])
                train_auc.append(train_roc)

            #appending to f1
            else:
                y_predict = logit_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                log_test_f1.append(f_score)
                f_train = f1_score(y_train, logit_model.predict(X_train))
                train_f1.append(f_train)
    
    return log_test_acc, log_test_auc, log_test_f1, train_acc, train_auc, train_f1

In [12]:
log_acc, log_auc, log_f1, log_train_acc, log_train_auc, log_train_f1 = LG_model(X_ohe, y)

Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   44.5s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   51.2s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:   41.7s remaining:    3.3s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   50.0s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.3s
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:   43.0s remaining:    3.4s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   51.8s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.2s
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:   40.4s remaining:    3.2s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   50.9s finished


Fitting 5 folds for each of 41 candidates, totalling 205 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.0s
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done 190 out of 205 | elapsed:   39.8s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done 205 out of 205 | elapsed:   48.7s finished


# Algorithm 2: KNN

In [13]:
def KNN_model(X, y):
    classifier = KNeighborsClassifier()
    num_k = np.ceil(np.logspace(0, 2.7, 3)).astype(int)
    
    search_space = {'n_neighbors': num_k, 
                      'weights': ['uniform', 'distance']}

    TRIALS = 5#DOING 5 TRIALS
    acc_scores = []#saving the accuracy for each trial on test set
    auc_scores = []#saving the roc auc for each trial on test set
    f1_scores = []#saving the f1 score for each trial on test set
    train_acc = []
    train_auc = []
    train_f1 = []
    
    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        
        # Create grid search
        clf = GridSearchCV(classifier, search_space, cv=StratifiedKFold(n_splits=5), 
                           scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                           verbose=1, n_jobs=-1)
        best_model = clf.fit(X_train, y_train)

        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1'])]

        model_params = [model_acc, model_auc, model_f1]

        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            k = params['n_neighbors']
            weight = params['weights']

            #training best models on validation set
            KNN_model = KNeighborsClassifier()
            KNN_model.set_params(n_neighbors=k, weights=weight)
            KNN_model.fit(X_train, y_train)#training on 5000 points

            #model == model_acc
            if index == 0:
                score = KNN_model.score(X_test, y_test)
                acc_scores.append(score)
                train_score = KNN_model.score(X_train, y_train)
                train_acc.append(train_score)
                
            #model_auc
            elif index == 1:
                roc_score = roc_auc_score(y_test, KNN_model.predict_proba(X_test)[:, 1])
                auc_scores.append(roc_score)
                train_roc = roc_auc_score(y_train, KNN_model.predict_proba(X_train)[:, 1])
                train_auc.append(train_roc)

            #appending to f1
            else:
                y_predict = KNN_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                f1_scores.append(f_score)
                f_train = f1_score(y_train, KNN_model.predict(X_train))
                train_f1.append(f_train)
    
    return acc_scores, auc_scores, f1_scores, train_acc, train_auc, train_f1

In [14]:
knn_acc, knn_auc, knn_f1, knn_train_acc, knn_train_auc, knn_train_f1 = KNN_model(X_ohe, y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.0s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.8s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.8s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.2s finished


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.4s finished


# Algorithm 3: Random Forest

In [15]:
def RF_model(X, y):
    classifier = RandomForestClassifier()
    max_features = [1, 2, 4, 6, 8, 12, 16, 20]
    
    for feature in max_features:
        if feature > len(X.columns):
            max_features.remove(feature)
            
    search_space = {'n_estimators': [1024], 'max_features': max_features}
    
    TRIALS = 5#DOING 5 TRIALS
    acc_scores = []#saving the accuracy for each trial on test set
    auc_scores = []#saving the roc auc for each trial on test set
    f1_scores = []#saving the f1 score for each trial on test set
    train_acc = []
    train_auc = []
    train_f1 = []
    
    for i in range(TRIALS):
        #sampling 5000 training size for k-fold
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=5000)
        
        # Create grid search
        clf = GridSearchCV(classifier, search_space, cv=StratifiedKFold(n_splits=5), 
                       scoring=['accuracy', 'roc_auc', 'f1'], refit=False,
                       verbose=2, n_jobs=-1)
        best_model = clf.fit(X_train, y_train)
        
        #top 3 best params model parameters
        model_acc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_accuracy'])]
        model_auc = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_roc_auc'])]
        model_f1 = best_model.cv_results_['params'][np.argmin(best_model.cv_results_['rank_test_f1'])]
        
        model_params = [model_acc, model_auc, model_f1]
        
        #gets the scores for each model and append them to corresponding array
        for index, params in enumerate(model_params):
            n_estimators = params['n_estimators']
            max_features = params['max_features']
            
            #train once more on entire validation set
            RF_model = RandomForestClassifier(n_estimators=n_estimators, 
                                                  max_features=max_features)
            RF_model.fit(X_train, y_train)
            
            #scoring on test set using multiple metrics
            #model == model_acc
            if index == 0:
                score = RF_model.score(X_test, y_test)
                acc_scores.append(score)
                train_score = RF_model.score(X_train, y_train)
                train_acc.append(train_score)
            
            #model_auc
            elif index == 1:
                roc_score = roc_auc_score(y_test, RF_model.predict_proba(X_test)[:, 1])
                auc_scores.append(roc_score)
                train_roc = roc_auc_score(y_train, RF_model.predict_proba(X_train)[:, 1])
                train_auc.append(train_roc)

            #appending to f1
            else:
                y_predict = RF_model.predict(X_test)#predictions on test set
                f_score = f1_score(y_test, y_predict)
                f1_scores.append(f_score)
                f_train = f1_score(y_train, RF_model.predict(X_train))
                train_f1.append(f_train)
                
    return acc_scores, auc_scores, f1_scores, train_acc, train_auc, train_f1

In [16]:
rf_acc, rf_auc, rf_f1, rftrain_acc, rftrain_auc, rftrain_f1= RF_model(X_le, y)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   17.6s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   17.5s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
/Users/garvinmo/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   17.3s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   16.3s finished


Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   16.5s finished


# Results

## Logistic Regression Results:

In [17]:
print("""TEST SET PERFORMANCE:""")
scores = pd.DataFrame([log_acc, log_auc, log_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(scores)
print()

logt_scores = pd.DataFrame([log_train_acc, log_train_auc, log_train_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
logt_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print("TRAIN SET PERFORMANCE:")
print(logt_scores)

TEST SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0

TRAIN SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0


## KNN Results:

In [18]:
print("""TEST SET PERFORMANCE:""")
k_scores = pd.DataFrame([knn_acc, knn_auc, knn_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
k_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(k_scores)
print()

print("TRAIN SET PERFORMANCE:")
kt_scores = pd.DataFrame([knn_train_acc, knn_train_auc, knn_train_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
kt_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(kt_scores)

TEST SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0

TRAIN SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0


## Random Forest Results:

In [19]:
print("TEST SET PERFORMANCE:")
rf_scores = pd.DataFrame([rf_acc, rf_auc, rf_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
rf_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(rf_scores)
print()
print("TRAIN SET PERFORMANCE:")
rft_scores = pd.DataFrame([rftrain_acc, rftrain_auc, rftrain_f1], 
                     columns= ["Trial 1", "Trial 2", "Trial 3", "Trial 4", "Trial 5"])
rft_scores.rename(index={0: "ACC", 1: "ROC", 2: "FSC"}, inplace=True)
print(rft_scores)

TEST SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0

TRAIN SET PERFORMANCE:
     Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
ACC      1.0      1.0      1.0      1.0      1.0
ROC      1.0      1.0      1.0      1.0      1.0
FSC      1.0      1.0      1.0      1.0      1.0
